In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
from email import utils

1) Log file is loaded

In [61]:
file = open("log.txt", "r") 
lines = file.readlines()
for l in range(3): print(lines[l][:-1])

156.15.13.173 [01/Jan/2018:11:13:49 +0100] "GET AddProduct.html HTTP/1.0" 200 2356
176.247.135.49 [01/Jan/2018:11:18:07 +0100] "GET Shop.html HTTP/1.0" 200 7466
176.247.135.49 [01/Jan/2018:11:19:05 +0100] "GET AddProduct.html HTTP/1.0" 200 2356


2) Cass "Request" parses and keeps data of a single requests. Complete the TODO, where a given "line" (string) should be parsed in order to dervie the ip of the host making the request, time of the request (in seconds), and target page (name).

In [72]:

class Request:
    def __init__(self, log):
        # TODO DO PARSING HERE
        for log_part in log.split():
            try:
                ip = re.findall(r'(?:\d{1,3}\.){3}\d{1,3}', log_part)
            except ValueError:
                pass
            else:
                self.ip = ip[0]
                break

        for log_part in log.split():
            if log_part[0] == '[':
                hours = int(log_part.split(':')[1])*3600
                minutes = int(log_part.split(':')[2])*60
                self.time = int(log_part.split(':')[3]) + minutes + hours
            elif log_part.endswith('.html'):
                self.page = log_part

        # convert hours + minutes + seconds -> total number of seconds
        
    def __repr__(self):
        return self.ip + "     " +  str(datetime.timedelta(seconds=self.time)) + " -> " + self.page

3) Each line of log.txt is parsed and the Request objects are added to the list

In [73]:
reqs = [Request(line) for line in lines]
for l in range(10): 
    print(reqs[l]) 

156.15.13.173     11:13:49 -> AddProduct.html
176.247.135.49     11:18:07 -> Shop.html
176.247.135.49     11:19:05 -> AddProduct.html
176.247.135.49     11:29:05 -> Summary.html
69.84.30.199     11:43:59 -> Shop.html
69.84.30.199     11:45:16 -> AddProduct.html
57.116.45.120     11:46:45 -> AddProduct.html
69.84.30.199     11:50:36 -> Summary.html
69.84.30.199     11:52:38 -> Payment.html
26.19.183.202     11:53:11 -> AddProduct.html


# Exercise

Your task is to analyze the log file. You should indentify the users and sessions. Then, compute the following: <br>
3.1) the number of sessions (should be 1000 :) ), <br>
3.2) the number of users, <br>
3.3) the average number of sessions per one user <br>
3.4) use the distribution plot to visualize the sessions' lengths (minutes);  <br>
3.5) use the distribution plot to visualize the sessions' lengths (the number of requests) <br>
3.6) use the distribution plot to visualize sessions' starting times (hours). <br>
3.7) which web page is the most common entry point?; for each page plot the number of times it has been an entry page,  <br>
3.8) which web page is the most common departure point?; for each page plot the number of times the user stopped a session while being on this page, <br>
3.9) for each page plot the average time the user stayed on this page <br>
3.10) print the fist 10 sessions (paths). <br>

You may find the following classes helpful and data.

In [183]:
class Session:
    def __init__(self):
        self.startTime = 0
        self.stopTime = 0
        self.requests = []
        
    def __repr__(self):
        print("SESSION")
        print(self.startTime)
        print(self.stopTime)
        STR = ""
        for r in self.requests:
            print(r)
            STR += ("  " + r.page + "    " + str(datetime.timedelta(seconds=r.time)) + "\n")
        return STR

class Computer:
    def __init__(self):
        self.ip = []
        self.sessions = []
    def __repr__(self):
        return self.ip

PAGES = ["Shop.html", "AddProduct.html", "Summary.html", "Payment.html","About.html", "Contact.html"]

LINKS = [[1, 4],
         [2,4],
         [3],
         [],
         [1,5],
         []]

######### USERS COUNT  #######################
users = {}
for req in reqs:
    req = str(req).split()
    req = [*req[:2], *[req[3]]]

    if req[0] not in users:
        users[req[0]] = []
    users[req[0]].append([req[1],req[2]])

######### SESSIONS COUNT  #######################
threshold = 765
session_count = 0
session_lengths = []
for i in users.keys():
    c = 1
    personal_count = 1
    session_start = -1
    session_end = -1
    for j in users[i][:-1]:
        k = users[i][c]
        j = j[0].split(':')
        k = k[0].split(':')

        hj = int(j[0])*3600
        mj = int(j[1])*60
        sj = int(j[2]) + mj + hj
        if session_start == -1:
            session_start = sj

        hk = int(k[0])*3600
        mk = int(k[1])*60
        sk = int(k[2]) + mk + hk

        if sk - sj > threshold:
            session_count += 1
            personal_count += 1
            session_end = sk
        c += 1
        if session_end != -1:
            session_lengths.append(round((session_end-session_start)/60.0,2))
            session_start = -1
            session_end = -1
    users[i].append(personal_count)
    session_count += 1
    session_lengths.append(round((sk-session_start)/60.0,2))
    session_start = -1
    session_end = -1

######### AVERAGE SESSIONS #######################
average_usr_sess = round(session_count/len(users.keys()),2)

print("3.1) The number of sessions: " + str(session_count))
print("3.2) The number of users: " + str(len(users.keys())))
print("3.3) The average number of sessions per one user: " + str(average_usr_sess))
print(session_lengths)

# plot distribution
# count, bins, ignored = plt.hist(data, 50, alpha=0.5, normed=True)
# plt.plot()

# x_axis = np.arange(len(pages))
# plt.bar(x_axis, data, align='center', alpha=0.5)
# plt.xticks(y_pos, pages, rotation=90)

3.1) The number of sessions: 1000
3.2) The number of users: 391
3.3) The average number of sessions per one user: 2.56
[234.78, 284.43, 7.58, 428.92, 11.32, 307.27, 67.23, 1.13, 48.53, 326.52, 1.35, 414.73, 1127.93, 442.93, 8.93, 1.22, 11.02, 293.7, 107.37, 12.13, 247.12, 145.23, 82.83, 0.93, 194.52, 84.9, 8.73, 169.85, 61.75, 71.03, 1.35, 254.67, 53.78, 1054.52, 153.8, 130.38, 1.0, 10.88, 309.3, 8.15, 277.28, 34.6, 1.35, 45.65, 32.32, 410.08, 9.15, 172.27, 44.72, 22.65, 102.17, 174.1, 138.43, 9.7, 83.72, 294.48, 13.25, 108.8, 154.48, 223.8, 18.25, 265.98, 41.38, 84.03, 7.87, 61.52, 133.48, 110.0, 1.58, 146.13, 223.27, 152.92, 1298.4, 170.8, 220.67, 1.67, 7.72, 153.48, 288.93, 1224.18, 159.7, 12.55, 159.62, 178.47, 9.68, 118.88, 0.92, 219.22, 162.48, 8.3, 245.2, 133.2, 12.3, 68.37, 179.9, 48.7, 1.08, 137.6, 197.68, 72.8, 13.55, 132.37, 9.97, 215.53, 31.28, 190.32, 1227.03, 360.48, 0.88, 231.18, 72.95, 53.82, 13.03, 251.23, 1043.93, 280.45, 61.25, 52.5, 1188.4, 148.93, 152.88, 6.68, 13.